In [ ]:
# 📦 Imports système
import sys
from pathlib import Path

# Accès au dossier racine du projet
root_dir = Path().resolve().parent

# Ajoute le dossier 'src/' au PYTHONPATH
sys.path.append(str(root_dir / "src"))

print("Ajouté au PYTHONPATH:", str(root_dir / "src"))

from pathlib import Path
print("Répertoire courant :", Path().resolve())
import os

print("Contenu de data/raw :")
for file in os.listdir("../data/raw"):
    print("-", file)


# Ensuite les imports Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit

# Et maintenant l'import fonctionne :
from features import add_length_features, build_tfidf


# 📂 Chargement des tweets JSON
# Lister les fichiers trouvés
files = list(Path("data/raw").glob("tweet_*.json"))
print("Fichiers trouvés :", files)

import json

# 📂 Chargement des tweets JSON
tweets = []

raw_dir = (Path("../data/raw")).resolve()
print("Chargement depuis :", raw_dir)

for f in raw_dir.glob("tweet_*.json"):
    print(f"Lecture de : {f.name}")
    with open(f, "r", encoding="utf-8") as file:
        data = json.load(file)

        # Cas : un seul tweet (dict)
        if isinstance(data, dict):
            tweets.append(pd.DataFrame([data]))

        # Cas : liste de tweets
        elif isinstance(data, list):
            tweets.append(pd.DataFrame(data))

# 🧾 Concaténation des DataFrames
if tweets:
    df = pd.concat(tweets, ignore_index=True)
    print(f"{len(df)} tweets chargés ✅")
else:
    raise ValueError("Aucun tweet trouvé. Vérifie le dossier data/raw.")


# 🧹 Nettoyage basique
df = df.dropna(subset=["text"]).drop_duplicates()

# 📊 Visualisation de base
df["length"] = df["text"].str.len()
df["word_count"] = df["text"].str.split().apply(len)

# ➕ Statistiques
print(df["label"].value_counts(normalize=True))
df.describe()

# 📈 Histogramme longueur
plt.figure(figsize=(6, 4))
sns.histplot(df["length"], bins=30)
plt.title("Distribution de la longueur des tweets")
plt.show()

# 📊 Répartition des classes
plt.figure()
sns.countplot(x="label", data=df)
plt.title("Toxicité des tweets")
plt.show()

# ✂️ Split train/val/test stratifié
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(df, df["label"]):
    train_val = df.iloc[train_idx]
    test = df.iloc[test_idx]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_idx, val_idx in split2.split(train_val, train_val["label"]):
    train = train_val.iloc[train_idx]
    val = train_val.iloc[val_idx]

print(f"Train : {len(train)}, Val : {len(val)}, Test : {len(test)}")

# 💾 Sauvegarde
Path("data/processed").mkdir(exist_ok=True, parents=True)
train.to_csv("data/processed/train.csv", index=False)
val.to_csv("data/processed/val.csv", index=False)
test.to_csv("data/processed/test.csv", index=False)

# 📐 TF-IDF sur train (demo)
X_train_tfidf, vectorizer = build_tfidf(train["text"])
print("TF-IDF shape :", X_train_tfidf.shape)


Ajouté au PYTHONPATH: C:\Users\aelma\Desktop\tweet-toxicity-detector\ml-production-template\src
Répertoire courant : C:\Users\aelma\Desktop\tweet-toxicity-detector\ml-production-template\notebooks
Contenu de data/raw :
- tweets.db
- tweet_0.json
- tweet_1.json
- tweet_2.json
- tweet_3.json
Fichiers trouvés : []


ValueError: If using all scalar values, you must pass an index